# ナイーブベイズ分類器
参考：データ分析のための機械学習入門 p98-113
https://isbn.sbcr.jp/88084.html

In [1]:
from collections import defaultdict

In [2]:
# ナイーブベイズ分類器
class NaiveBayesClassifier:

    # 保持すべき値を初期化する
    def __init__(self):
        self.alpha = 1
        self.total = 0
        self.categories = defaultdict(int) # ex. {'Good': 4, 'Bad': 4}
        self.features = defaultdict(lambda: defaultdict(int)) # ex. {'よい': {'Good': 3, 'Bad': 1}, '最悪': {'Bad': 2}}
    
    # 学習
    def fit(self, features, category):
        self.total += 1
        self.categories[category] += 1
        for f in features:
            self.features[f][category] += 1
        return
    
    # P(カテゴリ) = (カテゴリの出現回数+α) /  (総数+α*カテゴリ数)
    def p_category(self, category):
        score = self.categories[category] + self.alpha
        score /= (self.total + self.alpha * len(self.categories))
        return score

    # P(素性|カテゴリ) = (素性を含むカテゴリの出現回数+α) /  (カテゴリの出現回数+2α)
    def p_feature_category(self, feature, category):
        score = self.features[feature][category] + self.alpha
        score /= (self.categories[category] + self.alpha * 2)
        return score

    # 適用
    def predict(self, features):
        max_score = -1.0
        max_category = None
        
        for c in self.categories:

            # カテゴリの確率
            score = self.p_category(c)
            
            # カテゴリにおける各素性の条件付き確率
            for f in self.features:
                p = self.p_feature_category(f, c)
                score *= p if f in features else (1 - p) # 書籍に誤植あり、修正済み
            
            #  スコアが最大となるカテゴリを採択する
            if score > max_score:
                max_score = score
                max_category = c
        
        return max_category, max_score

In [3]:
# 学習用データ
training_data = [
    [['よい', 'とても'], 'Good'],
    [['よい', 'とても', 'すばらしい'], 'Good'],
    [['よい', 'すばらしい', '見つかりません'], 'Good'],
    [['すばらしい'], 'Good'],
    [['見つかりません', '買いたくない'], 'Bad'],
    [['よい'], 'Bad'],
    [['とても', '最悪', '買いたくない'], 'Bad'], # 書籍に誤植あり、修正済み
    [['最悪'], 'Bad'],
]
test_data = [
    ['よい', 'とても'],
    ['とても', 'すばらしい', '買いたくない'],
    ['最悪']
]

In [4]:
# インスタンス作成
nbc = NaiveBayesClassifier()

# 学習
for f, c in training_data:
    nbc.fit(f, c)

In [5]:
# 学習結果の確認

# 各カテゴリの確率
for c in nbc.categories:
    print('p(%s) = %f' % (c, nbc.p_category(c)))

# 各カテゴリにおける各素性の条件付き確率
for f in nbc.features:
    for c in nbc.categories:
        print('p(%s, %s) = %f' % (f, c, nbc.p_feature_category(f, c)))

p(Good) = 0.500000
p(Bad) = 0.500000
p(よい, Good) = 0.666667
p(よい, Bad) = 0.333333
p(とても, Good) = 0.500000
p(とても, Bad) = 0.333333
p(すばらしい, Good) = 0.666667
p(すばらしい, Bad) = 0.166667
p(見つかりません, Good) = 0.333333
p(見つかりません, Bad) = 0.333333
p(買いたくない, Good) = 0.166667
p(買いたくない, Bad) = 0.500000
p(最悪, Good) = 0.166667
p(最悪, Bad) = 0.500000


In [6]:
# 適用
for f in test_data:
    print(f, nbc.predict(f))

['よい', 'とても'] ('Good', 0.025720164609053502)
['とても', 'すばらしい', '買いたくない'] ('Good', 0.0051440329218107005)
['最悪'] ('Bad', 0.03086419753086421)


In [7]:
# α=0だと出現回数が少ない素性において極端な判定結果となる
# α=1などで分母と分子の値にゲタを履かせることで精度が上がる
nbc.alpha = 0
for f in test_data:
    print(f, nbc.predict(f))

['よい', 'とても'] ('Good', 0.03515625)
['とても', 'すばらしい', '買いたくない'] ('Good', 0.0)
['最悪'] ('Bad', 0.052734375)
